#### NX-414: Brain-like computation and intelligence
##### TA: Alessandro Marin Vargas

# Best model usage - Mini projects (Predicting neural activity)

## Import the relevant packages and functions

In [12]:
from utils import load_it_data
import torch
from torchvision.models import ResNet, resnet50, ResNet50_Weights
import joblib
import torch.nn as nn
import numpy as np

## Load the data

In [13]:
path_to_data = '' ## Insert the folder where the data is, if you download in the same folder as this notebook then leave it blank

stimulus_train, stimulus_val, stimulus_test, objects_train, objects_val, objects_test, spikes_train, spikes_val = load_it_data(path_to_data)

## Load the models
Load the models that have been saved in joblib format (and add the function that extract the activations to the ResNet model).

In [14]:
num_classes = len(np.unique(objects_train))
resnet = resnet50(weights=ResNet50_Weights.DEFAULT)
resnet.fc = nn.Linear(resnet.fc.in_features, num_classes)
params_to_update = resnet.parameters()
resnet.load_state_dict(torch.load('resnet50_finetuned.pt'))

pca = joblib.load('pca_layer3.joblib')
ridge = joblib.load('ridge_layer3.joblib')

RuntimeError: Attempting to deserialize object on a CUDA device but torch.cuda.is_available() is False. If you are running on a CPU-only machine, please use torch.load with map_location=torch.device('cpu') to map your storages to the CPU.

# Extract the activations from the finetuned ResNet model
Define the function that extract the activations from the layer 3 and add it to the ResNet model.

In [22]:
# Initialize and reshape the model
num_classes = len(np.unique(objects_train))
model = resnet50(weights=ResNet50_Weights.DEFAULT)
model.fc = nn.Linear(model.fc.in_features, num_classes)
params_to_update = model.parameters()
model.load_state_dict(torch.load('resnet50_finetuned.pt'))

<All keys matched successfully>

In [23]:
def extract_activations(self, stimuli) :
    """extract the activations of the model for the given stimuli

    Args:
        model (model): model we want to extract the activations from
        stimuli (ndarray): input data of the processed image's pixels

    Returns:
        list of ndarray: list of activations for each stimulus
    """    
    activations = []
    for x in stimuli : 
        x = self.conv1(x.unsqueeze(0))
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = self.layer1(x)
        x = self.layer2(x)        
        x = self.layer3(x)
        activations.append(torch.flatten(x.squeeze(0)).detach().cpu().numpy())
    return activations

ResNet.extract_activations = extract_activations

In [25]:
activations_val = model.extract_activations(torch.tensor(stimulus_val))

# Compute the 1000 PCs from the activations

In [26]:
activations_val = pca.transform(activations_val)

# Make predictions

In [29]:
predictions = ridge.predict(activations_val)